In [11]:
import rl
import gym
import gym_search

envs = gym.vector.make("Gaussian-v0", num_envs=1, shape=(15, 15))
map_agent = rl.agents.make("map", envs, use_position=True)
lstm_agent = rl.agents.make("lstm", envs)

def num_params(model, trainable=True):
    if trainable:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

print(num_params(map_agent))
print(num_params(lstm_agent))

print(map_agent.map_net.output_dim)

879750
945574
64
